# ORE Dashboard

Some introduction etc...

## Launch ORE

In [ ]:
from ipywidgets import Text, Button
from IPython.display import display
import subprocess
import os

ore_path_selector = Text(description='ORE Path:', value='D:\\cppdev\\openxva\\App\\bin\\x64\\Debug\\', width=200)
xml_selector = Text(description='XML File:', value='D:\cppdev\openxva\Examples\Example_2\Input\\ore.xml', width=200)
launch_button = Button(description='Launch')

npv_cube_filename = ""

def launch_ore(b):
    cwd = os.getcwd() # save directory of the jupyter notebook
    print(cwd)
    config_path = os.path.dirname(os.path.dirname(xml_selector.value))
    os.chdir(config_path) # navigate to ore config folder
    print(os.getcwd())
    command = [ore_path_selector.value + 'ore.exe', os.path.join(os.path.join(config_path, 'Input'),'ore.xml')]
    print(command)
    print("Starting ORE run... please wait...")
    p = subprocess.Popen(command,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE)
    stdout, stderr = p.communicate()
    print(stdout)
    print(stderr)
    cwd = os.getcwd()
    os.chdir(cwd) # go back to jupyter notebook
    print("ORE run successful!")
    npv_cube_filename = os.path.join(os.path.join(config_path, 'Output'),'rawcube.csv')

launch_button.on_click(launch_ore)

display(ore_path_selector)
display(xml_selector)
display(launch_button)

In [ ]:
print(os.getcwd())

## Select Netting Sets and Trades

In [ ]:
import pandas as pd
import numpy as np

class NpvCube(object):

    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file)
        self.df.columns = ['Id', 'NettingSet', 'DateIndex' , 'Date', 'Sample', 'Value']
        self.dates = self.df.Date.unique()
        self.trades = self.df.Id.unique()
        self.netting_sets = self.df.NettingSet.unique()
        self.datedict = {val: num for num, val in enumerate(self.dates)}
        self.tradedict = {val: num for num, val in enumerate(self.trades)}
        self.netting_set_dict = {val: num for num, val in enumerate(self.netting_sets)}
        self.nd = np.array([])
        self.density_values = np.array([])
        self.dist_space = np.array([])
        self.exposure_statistics ={}
        self.npv_min = None
        self.npv_max = None
        self.density_max = None
        self.grid_size = 50
        #self.calc_nd()

    def calc_nd(self):
        """ Converts DataFrame into 3D array. """
        self.nd = np.zeros((len(self.dates), len(self.trades), self.df.Sample.max() + 1))
        for row in self.df.values:
            i = self.datedict[row[2]]
            j = self.tradedict[row[0]]
            k = row[3]
            self.nd[i][j][k] = row[4]

    def trades_samples_per_date(self, trades):
        # ToDo: this can probably be improved
        sum = np.array([self.nd[self.datedict[date], trades[0], :] for date in self.dates])
        for i in range(1, len(trades)):
            sum += np.array([self.nd[self.datedict[date], trades[i], :] for date in self.dates])
        return sum

    def calc_density_data(self, trades, exposure):
        data = self.trades_samples_per_date(trades)
        if exposure == ExposureTypes.npvp:
            data = np.fmax(data, np.zeros(data.shape))
        elif exposure == ExposureTypes.npvm:
            data = np.fmax(-data, np.zeros(data.shape))
        self.dist_space = np.linspace(np.min(data), np.max(data), self.grid_size)
        self.density_values = np.zeros((len(self.dates), self.grid_size))
        for k in range(len(data)):
            row = data[k]
            try:
                density = gaussian_kde(row)
                self.density_values[k] = density(self.dist_space)
            except LinAlgError:
                self.density_values[k] = np.zeros(self.grid_size)
        self.density_max = np.max(self.density_values)
        self.npv_min = np.amin(data)
        self.npv_max = np.amax(data)
        if exposure == ExposureTypes.npvp:
            self.npv_min = max(self.npv_min, 0)
            self.npv_max = max(self.npv_max, 0)
        elif exposure == ExposureTypes.npvm:
            self.npv_min = min(self.npv_min, 0)
            self.npv_max = min(self.npv_max, 0)

    def calc_exposure_statistics(self, trade):
        data = self.trades_samples_per_date(trade)
        data_plus = np.fmax(data, np.zeros(data.shape))
        data_minus = -np.fmax(-data, np.zeros(data.shape))
        npv = np.average(data, axis=1)
        npv_plus = np.average(data_plus, axis=1)
        npv_minus = np.average(data_minus, axis=1)
        perc = np.percentile(data, 95 , axis=1)
        self.exposure_statistics = {
            'npv': npv,
            'npv+': npv_plus,
            'npv-': npv_minus,
            'perc': perc}

In [ ]:
print(npv_cube_filename)

import pandas as pd       

from ipywidgets import SelectMultiple, HBox

current_netting_sets = [0]
def set_current_netting_sets(netting_sets):
    current_netting_sets = netting_sets['new']

current_trades = [0]
def set_current_trades(trades):
    current_trades = trades['new']
    
df = None
netting_set_selector = SelectMultiple(description='Netting Sets:')
trades_selector = SelectMultiple(description='Trades:')
netting_set_selector.observe(set_current_netting_sets, names='value')
trades_selector.observe(set_current_trades, names='value')
def load_cube_file(b):
    csv_file = npv_file_selector.value
    df = pd.read_csv(csv_file)
    df.columns = ['Id', 'NettingSet', 'DateIndex' , 'Date', 'Sample', 'Value']
    tradedict = {val: num for num, val in enumerate(df.Id.unique())}
    netting_set_dict = {val: num for num, val in enumerate(df.NettingSet.unique())}
    netting_set_selector.options = netting_set_dict
    trades_selector.options = tradedict

npv_file_selector = Text(description='Cube File:', value='D:\cppdev\openxva\Examples\Example_2\Output\\rawcube.csv', width=200)
read_npv_button = Button(description='Load')
read_npv_button.on_click(load_cube_file)

display(HBox([npv_file_selector, read_npv_button]))
display(HBox([netting_set_selector, trades_selector]))

## NPV Surface

In [ ]:
%matplotlib notebook
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from matplotlib import cm
from enum import Enum
from matplotlib.font_manager import FontProperties

class ExposureTypes(Enum):
    npv = 'NPV'
    npvp = 'NPV+'
    npvm = 'NPV-'
    
current_exposure=ExposureTypes.npv

npv.calc_density_data(current_trades, current_exposure)
fig_surface = plt.figure()
ax_surface =fig_surface.add_subplot(111, projection='3d')

date_step = 5
ax_surface.set_xticks(np.arange(0, len(npv.dates), date_step))
ax_surface.set_xticklabels(npv.dates[0::date_step])
ax_surface.set_xlabel('Dates')
ax_surface.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
ax_surface.set_ylabel(current_exposure.value)

X, Y = np.meshgrid(np.arange(len(npv.dates)), npv.dist_space)
ax_surface.plot_trisurf(X.flatten(), Y.flatten(), npv.density_values.T.flatten(), cmap=cm.jet, linewidth=0)
plt.show()

## Time Slider

In [ ]:
from ipywidgets import IntSlider

current_exposure=ExposureTypes.npv
npv.calc_density_data(current_trades, current_exposure)

fig_surface = plt.figure()
ax_time_slider = fig_surface.add_subplot(111)
fig_surface.canvas.set_window_title('Time Slider')
            
def plot_time_slider(change):
    date = change['new']
    plt.cla()
    ax_time_slider.plot(npv.dist_space, npv.density_values[date], color='k', label=current_exposure.value)
    ax_time_slider.set_xlim(npv.npv_min, npv.npv_max)
    ax_time_slider.set_ylim(0, npv.density_max)
    
time_slider = IntSlider(min=0, max=len(npv.dates)-1, value=0, description='DateIndex:')
time_slider.observe(plot_time_slider, names='value')
display(time_slider)


## Exposure Statistics

In [ ]:
npv.calc_exposure_statistics(current_trades)
fig_statistics = plt.figure()
ax_statistics = fig_statistics.add_subplot(111)
ax_statistics.plot(range(len(npv.dates)), npv.exposure_statistics['npv'], label='NPV')
ax_statistics.plot(range(len(npv.dates)), npv.exposure_statistics['npv+'], label='NPV+')
ax_statistics.plot(range(len(npv.dates)), npv.exposure_statistics['npv-'], label='NPV-')
ax_statistics.plot(range(len(npv.dates)), npv.exposure_statistics['perc'], label='perc')
ax_statistics.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
date_step = 2
ax_statistics.set_xticks(np.arange(0, len(npv.dates), date_step))
ax_statistics.set_xticklabels(npv.dates[0::date_step], rotation=90)
fontP = FontProperties() # set legend outside plot
fontP.set_size('small')
ax_statistics.legend(loc='upper right', shadow=True, prop=fontP)
plt.show()